In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
# Carregando dados
# Lista de Umbilicais
dcable = pd.read_excel("DATA_CABLE.xlsx")

In [3]:
triangular_membership_function = {1:[1,1,1] , 2:[1,2,3] , 3:[2,3,4] , 4:[3,4,5] , 5:[4,5,6] , 6: [5,6,7] , 7:[6,7,8],8:[7,8,9],9:[9,9,9]}
#test_data = [[1,5,4,7],[0.2,1,0.5,3],[0.25,2,1,3],[0.142,0.33,0.33,1]]

def fuzzy_AHP(AHP_matrix):
	#print(triangular_membership_function)
	test_data = AHP_matrix
	n = len(test_data)
	fuzzified_test_data = np.zeros((n,n,3))

	for x in range(n):
		for y in range(n):
			if(test_data[x][y] >= 1):
				fuzzified_test_data[x][y] = triangular_membership_function[test_data[x][y]]
			else:
				index = round(1/test_data[x][y])
				#print(index)
				temp = triangular_membership_function[index]
				for i in range(3):
					fuzzified_test_data[x][y][i] = 1.0/temp[2-i]
	#print(fuzzified_test_data)

	fuzzy_geometric_mean = [[1 for x in range(3)] for y in range(n)]
	#print(fuzzy_geometric_mean)

	for i in range(n):
		for j in range(3):
			for k in range(n):
				fuzzy_geometric_mean[i][j] *= fuzzified_test_data[i][k][j]
			fuzzy_geometric_mean[i][j] = fuzzy_geometric_mean[i][j]**(1/float(n))
	#print(fuzzy_geometric_mean)

	sum_fuzzy_gm = [0 for x in range(3)]
	inv_sum_fuzzy_gm = [0 for x in range(3)]

	for i in range(3):
		for j in range(n):
			sum_fuzzy_gm[i] += fuzzy_geometric_mean[j][i]

	for i in range(3):
		inv_sum_fuzzy_gm[i] = (1.0/sum_fuzzy_gm[2-i])
	#print(sum_fuzzy_gm)

	fuzzy_weights = [[1 for x in range(3)] for y in range(n)]

	for i in range(n):
		for j in range(3):
			fuzzy_weights[i][j] = fuzzy_geometric_mean[i][j]*inv_sum_fuzzy_gm[j]
	#print(fuzzy_weights)

	weights = [0 for i in range(n)]
	normalized_weights = [0 for i in range(n)]
	sum_weights = 0

	for i in range(n):
		for j in range(3):
			weights[i] += fuzzy_weights[i][j]
		weights[i] /= 3
		sum_weights += weights[i]
	#print(weights)
	#print(sum_weights)

	for i in range(n):
		normalized_weights[i] = (1.0*weights[i])/(1.0*sum_weights)
	#print(normalized_weights)

	return normalized_weights

#if __name__=="__main__":
#	main()

In [54]:

def Fluxo(D,Vt,Vcabo,FP,P, EFF):
  # Inicio do algoritmo
  # Dados Elétricos
  S = P/(FP*EFF) # Potência aparente da Carga
  I=(S/Vt/(3)**0.5)*1000 # Corrente da carga
  
  #Dados para calculo de custo de instalação
  ton_fac = 300 # peso em tonelada na bobina nas fábricas
  ton_PLSV = 3640 # peso em tonelada no PLSV
  viagem = 2 # numero de dias de viagem
  t_emendas = 1.5 # tempo em dias para emendas
  vel_SPOOL = 17.2 # velocidade km/dia para spool
  vel_PLSV = 8.6 # velocidade km/dia do PLSV
  diaria_PLSV = 275 # custo da diaria do PLSV em U$M

  # Lista de Cabos dependendo da classe de tensão
  # Opção 12/20 kV
  if Vt < 20:
    list_cable = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
    Vsmax = Vcabo*1000*0.9 # Tensão máxima de operação
  # Opção 18/30 kV
  elif Vt > 20 and Vt < 36:
    list_cable = [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
    Vsmax = Vcabo*1000*0.9 # Tensão máxima de operação
  # Opção 36/69 kV
  elif Vt > 36 and Vt < 69:
    list_cable = [24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]
    Vsmax = Vcabo*1000*0.9 # Tensão máxima de operação
  # Opção 76/138 kV
  elif Vt > 70:
    list_cable = [36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
    Vsmax = Vcabo*1000*0.9 # Tensão máxima de operação


  # Escolha do Cabo
  n = 1 # número de cabos por fase
  i = list_cable[0] # inicio da lista de cabos para algoritmo 
  #t = len(list_cable)-1
  #print(t)
  t = list_cable[11] # ffinal da lista de cabos para algoritmo 
  fim = 0
  # Laço para escolha de cabo
  # Inicio com a menor bitola e tenta atender as premissas
  while n < 4:
    num = n
    i = list_cable[0]
    #print(n)
    while  i <= t:
      #print(i)
      R = dcable.loc[i, 'Rca'] # Resistência do cabo Ohms/km
      Xl = dcable.loc[i, 'XL'] # Resistência do cabo Ohms/km
      Xc = dcable.loc[i, 'XC'] # Resistência do cabo Ohms*km
      Icabo1 = dcable.loc[i, 'I-55']  # Corrente nominal a 55°C
    # Fator de agrupamento dependendo do n° de cabos por fase
      if num == 1:
        fcorr = 1
      elif num == 2:
        fcorr = 0.9
      else:
        fcorr = 0.9 

      Icabo = (Icabo1/1.20)*num*fcorr #Corrente nominal com fator de 20%
      
    # Cálculo de Impedância equivalente do Cabo
      Rc = R*D/num
      Xlc = Xl*D/num
      Yc = (1/Xc)*D*num
      ZZ=complex(Rc,Xlc)
      ZZZ=complex(0, Yc)
      Zc=(ZZ/ZZZ)**0.5
      Yl=(ZZ*ZZZ)**0.5
      Zeq = np.sinh(Yl)*Zc
      a = (np.cosh(Yl)-complex(1,0))/Zeq

      # Fluxo de Potência
      Vr = complex(Vt/(3)**0.5*1000,0) # Tensão de saída do umbilical
      Vr3 = Vr*3**0.5 # Tensão trifásica de saída do umbilical
      Ir = complex(I*FP*EFF,-I*(1-(FP*EFF)**2)**0.5) # Corrente de saída do umbilical
      dV = (Vr*a+Ir)*Zeq # Queda de tensão no umbilical
      Vs = Vr+dV # Tensão de entrada do umbilcal
      Vs3 = Vs*3**0.5 # Tensão trifásica de entrada do umbilcal
      Is = (dV/Zeq)+(Vs*a)  # Corrente de entrada do umbilcal
      Ps = Vs*complex.conjugate(Is)  # Potência de entrada do umbilcal
      realPs = Ps.real*3/1000000  # Potência ativa de entrada do umbilcal
      Vsp = (abs(Vs)/Vt/1000)*(3)**0.5*100 # Relação Vs/Vt
      Isp = (abs(Is)/Icabo)*100 # Relação Is/Vr
      Vrp = abs(Vr)/Vt/1000*(3)**0.5*100 # Relação Vr/Vt
      Irp = abs(Ir)/Icabo*100 # Relação Is/Ir
      PerdW = realPs-P # Perdas ao longo do umbilical
      Perd = (PerdW)/P*100 # Relação das Perdas/Potência ativa da carga
      
      
      # Custos
      # Custo de Aquisição dos cabos
      Custo_Aq = dcable.loc[i, 'Custo']*D*num/1000
      # Custo de Instalação dos cabos  
      Peso_umb = dcable.loc[i, 'Peso']*D
      dias_viagem = math.ceil(Peso_umb/ton_PLSV)*viagem
      n_emendas = math.floor(Peso_umb/ton_fac)*num
      dias_emendas = math.ceil(n_emendas*t_emendas)
      dias_spool = math.ceil(D*num/vel_SPOOL)
      dias_inst = math.ceil(D*num/vel_PLSV)
      dias_total = dias_viagem + dias_emendas + dias_spool + dias_inst
      Custo_Ins = dias_total*diaria_PLSV/1000



      #print(dcable.loc[i, 'Bitola'],Vsp,Isp,abs(Vs3), Perd, Custo_Aq)
      
      # Condição para atendimento das premissas
      # Deve atender Is e Ip menor que Icabo com 20% de segurança
      # Deve atender no máximo de 10% de queda de tensão
      # Deve atender no máximo de 7% de Perdas
      if Isp < 100 and Irp <100 and Vsp < 110 and Perd < 7 and abs(Vs3) < Vsmax:
        #Resultado = {'Bitola': ["%.2f" % dcable.loc[i, 'Bitola'],num, "%.2f" % Icabo],'Vs': ["%.2f" %abs(Vs3),"%.2f" %Vsp],'Is': ["%.2f" %abs(Is),"%.2f" %Isp],'Ir': ["%.2f" %abs(Ir),"%.2f" %Irp],'Vr': ["%.2f" %abs(Vr3),"%.2f" %Vrp], 'Perdas': ["%.2f" %PerdW, "%.2f" %Perd ]}
        Resultado = {'D':[D],
                     'Ps':[round(abs(Ps*3/1000000),2)],
                     'Bitola': [round(dcable.loc[i, 'Bitola'],0)],
                     'num':[num],'Vs': [round(abs(Vs3),2)],
                     'Vs%':[round( Vsp,2)],
                     'Is': [round(abs(Is),2)],
                     'Is%':[round(Isp,0)],
                     'Ir': [round(abs(Ir),2)],
                     'Ir%':[round(Irp,0)],
                     'Vr': [round(abs(Vr3),2)],
                     'Vr%':[round(Vrp,2)],
                     'Perdas': [round(PerdW,2)],
                     'Perd%':[round(Perd,2) ],
                     'Preal':[Ps.real],
                     'Pimag':[Ps.imag],
                     'Custo_Aq':[round(Custo_Aq,2)],
                     'Custo_Ins':[Custo_Ins]}
        #s = pd.DataFrame(Resultado)
        fim = 1
        #print (s)
        break
      i = i + 1 
    
    if fim == 1:
      return pd.DataFrame(Resultado)
      break
    n = n +1
    
    

In [5]:
dt = pd.read_excel("dist.xlsx")
dt['D'] = ((dt['D_SWGx']-dt['Dx'])**2+(dt['D_SWGy']-dt['Dy'])**2)**0.5
dt

,D_SWGx,D_SWGy,Dx,Dy,Tipo,nu,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,D
0,29.761486,4.500000,30.000000,9.000000,1,1,NaN,4.506316,NaN,0,4.506316
1,29.761486,4.500000,30.000000,8.000000,1,2,NaN,3.508117,NaN,1,3.508117
2,29.761486,4.500000,30.000000,7.000000,1,3,NaN,2.511352,NaN,2,2.511352
3,29.761486,4.500000,30.000000,6.000000,1,4,NaN,1.518844,NaN,3,1.518844
4,29.761486,4.500000,30.000000,5.000000,1,5,NaN,0.553975,NaN,4,0.553975
...,...,...,...,...,...,...,...,...,...,...,...
149,29.329634,2.568593,30.000000,2.000000,12,150,NaN,0.879027,NaN,149,0.879027
150,0.000000,4.500000,29.329634,2.568593,12,151,NaN,29.393158,NaN,150,29.393158
151,29.325440,4.499728,30.000000,5.000000,12,152,NaN,0.839823,NaN,151,0.839823
152,29.325440,4.499728,30.000000,4.000000,12,153,NaN,0.839499,NaN,152,0.839499


In [71]:
dconf = pd.read_excel("Conf.xlsx")
dconf

,I%,R_Eq,R_Umb
0,0.0,0.999969,0.999988
1,1.0,0.999939,0.999939
2,2.0,0.999939,0.999939
3,3.0,0.999939,0.999939
4,4.0,0.999939,0.999939
...,...,...,...
96,96.0,0.749300,0.922918
97,97.0,0.718498,0.912848
98,98.0,0.684218,0.899950
99,99.0,0.646194,0.883648


In [132]:
# DADOS DE ENTRADA: Carga e Sistema

'''
n = 1  -> 1 painel com 10 alimentadores
n = 2  -> 2 paineis com 8 e 2 alimentadores
n = 3  -> 2 paineis com 7 e 3 alimentadores
n = 4  -> 2 paineis com 6 e 4 alimentadores
n = 5  -> 2 paineis com 5 e 5 alimentadores
n = 6  -> 3 paineis com 6, 2 e 2 alimentadores
n = 7  -> 3 paineis com 5, 3 e 2 alimentadores
n = 8  -> 3 paineis com 4, 4 e 2 alimentadores
n = 9  -> 3 paineis com 4, 3 e 3 alimentadores
n = 10 -> 4 paineis com 2, 2, 2 e 4 alimentadores
n = 11 -> 4 paineis com 2, 2, 3 e 3 alimentadores
n = 12 -> 5 paineis com 2, 2, 2, 2 e 2 alimentadores

  Este algoritmo executa as seguintes tarefas:
  - Calculo do fluxo de potência de cada circuito;
  - Criação da tabela com todas as alternativas;
  - Somatória das potências de número de circuitos de cada painel;
  - Cálculo da média de tensão de cada painel;
  - Cálculo do FP de cada painel;
  - Criação da tabela com todos os alimentadores e os paineis;
  - Criação da tabela dr, com as informações de Perdas e Potências de cada alternativa.
'''
for n in range(1,13):
 # n == j
  Vt =  25 # Tensão na carga
  Vcabo = 30
  FP = 0.92 # FP da carga
  EFF = 1
  P = 4 # Potência ativa da carga
  b = pd.DataFrame() 
  c = pd.DataFrame()
  d = pd.DataFrame()
  e = pd.DataFrame()
  f = pd.DataFrame()  
  if n == 1:
    for i in range(10):
      D = dt.loc[i,'D'] # busca a Distância das cargas
      aux = Fluxo (D, Vt,Vcabo,FP, P, EFF) # Calcula o fluxo de potência
      b = b.append(aux, ignore_index=True) # Adiciona uma linha nos resultado
    P = (b['Preal'].sum())*3/1000000 # Soma a potência de todas as 10 cargas
    Vt = b['Vs'].mean()/1000 # Calcula a Tensão média no Painel
    FP = math.cos(math.atan(b['Pimag'].mean()/1000/b['Preal'].mean()/1000)) # Calcula o Fator de potência no Painel
    D = dt.loc[10,'D'] # busca a Distância do Painel 
    aux1 =  Fluxo (D, Vt,Vcabo,FP, P, EFF) # Calcula o fluxo de potência
    b = b.append(aux1, ignore_index=True) # Adiciona uma linha nos resultado
    swg10 = b # Renomeia para a alternativa correspondente
    swg10
    # Criação da tabela a ser utilizada com Potência e Perdas
    P10 = swg10[swg10["Perd%"] < swg10["Perd%"].max()]['Perd%'].mean()+swg10['Perd%'].max() # Calcula a perda total
    Ps10 = swg10['Ps'].max() # Busca a potência da alternativa
    Pca10 = swg10['Custo_Aq'].sum()
    Pci10 = swg10['Custo_Ins'].sum()
    Pct10 = Pca10 + Pci10
    # Calculo da Confiabilidade
    P10RReq = dconf.loc[swg10[swg10["Perd%"] < 3]['Is%'].max(), 'R_Eq']**6
    P10RRum = dconf.loc[swg10[swg10["Perd%"] < 3]['Is%'].max(), 'R_Umb']
    P10RR = P10RReq*P10RRum
    P10RTeq = dconf.loc[swg10[swg10["Perd%"] > 3]['Is%'].max(), 'R_Eq']**3
    P10RTum = dconf.loc[swg10[swg10["Perd%"] > 3]['Is%'].max(), 'R_Umb']
    P10RT = P10RTeq*P10RTum
    P10R = P10RR*P10RT
    P10P = (1-P10RT)
    # Criação da Tabela
    dr = pd.DataFrame({'Tipo':['10'],
                          'Ps':[Ps10],
                          'Perd%':[P10],
                          'Cust_Aq':[Pca10],
                          'Custo_Ins':[Pci10],
                          'Custo_T':[Pct10],
                          'Conf_S%': [round(P10R,2)],
                          'Prob_10': [round(P10P,4)]})

    dr 

  elif n == 2:
    for i in range(11,19):
      D = dt.loc[i,'D'] # Distância
      aux = Fluxo (D, Vt,Vcabo,FP, P, EFF)
      b = b.append(aux, ignore_index=True)
    P = (b['Preal'].sum())*3/1000000
    Vt = b['Vs'].mean()/1000
    FP = math.cos(math.atan(b['Pimag'].mean()/1000/b['Preal'].mean()/1000))
    D = dt.loc[19,'D'] # Distância
    aux1 =  Fluxo (D, Vt,Vcabo,FP, P, EFF)
    b = b.append(aux1, ignore_index=True)
    for i in range(20,22):
      D = dt.loc[i,'D'] # Distância
      Vt =  25 # Tensão na carga
      Vcabo = 30
      FP = 0.92 # FP da carga
      EFF = 1
      P = 4 # Potência ativa da carga
      aux2 = Fluxo (D, Vt,Vcabo,FP, P, EFF)
      b = b.append(aux2, ignore_index=True)
      c = c.append(aux2, ignore_index=True)
    P = (c['Preal'].sum())*3/1000000
    Vt = c['Vs'].mean()/1000
    FP = math.cos(math.atan(c['Pimag'].mean()/1000/c['Preal'].mean()/1000))
    D = dt.loc[22,'D'] # Distância
    aux3 =  Fluxo (D, Vt,Vcabo,FP, P, EFF)
    b = b.append(aux3, ignore_index=True)  
    swg8x2 = b
    swg8x2
    P8x2x = swg8x2[swg8x2["Perd%"] < 3]['Perd%'].mean()
    P8x2y = swg8x2[swg8x2["Perd%"] > 3]['Perd%'].mean()
    P8x2 = (P8x2x + P8x2y)
    Ps8x2 = swg8x2[swg8x2['Ps'] > 8]['Ps'].sum()
    Pca8x2 = swg8x2['Custo_Aq'].sum()
    Pci8x2 = swg8x2['Custo_Ins'].sum()
    Pct8x2 = Pca8x2 + Pci8x2
    # Calculo da Confiabilidade
    P8x2RReq = dconf.loc[swg8x2[swg8x2["Perd%"] < 3]['Is%'].max(), 'R_Eq']**6
    P8x2RRum = dconf.loc[swg8x2[swg8x2["Perd%"] < 3]['Is%'].max(), 'R_Umb']
    P8x2RR = P8x2RReq*P8x2RRum
    P8x2RTeq = dconf.loc[swg8x2[swg8x2["Perd%"] > 3]['Is%'].max(), 'R_Eq']**3
    P8x2RTum = dconf.loc[swg8x2[swg8x2["Perd%"] > 3]['Is%'].max(), 'R_Umb']
    P8x2RT = P8x2RTeq*P8x2RTum
    P8x2R = P8x2RR*P8x2RT
    P8x2P = (1-P8x2RT)**2
    # Criação da Tabela
    dr8x2 = pd.DataFrame({'Tipo':['8x2'],
                          'Ps':[Ps8x2],
                          'Perd%':[P8x2],
                          'Cust_Aq':[Pca8x2],
                          'Custo_Ins':[Pci8x2],
                          'Custo_T':[Pct8x2],
                          'Conf_S%': [round(P8x2R,2)],
                          'Prob_10': [round(P8x2P,4)]})
    dr = dr.append(dr8x2, ignore_index=True)
    dr
  elif n == 3:
    for i in range(23,26):
      D = dt.loc[i,'D'] # Distância
      aux = Fluxo (D, Vt,Vcabo,FP, P, EFF)
      b = b.append(aux, ignore_index=True)
    P = (b['Preal'].sum())*3/1000000
    Vt = b['Vs'].mean()/1000
    FP = math.cos(math.atan(b['Pimag'].mean()/1000/b['Preal'].mean()/1000))
    D = dt.loc[26,'D'] # Distância
    aux1 =  Fluxo (D, Vt,Vcabo,FP, P, EFF)
    b = b.append(aux1, ignore_index=True)
    for i in range(27,34):
      D = dt.loc[i,'D'] # Distância
      Vt =  25 # Tensão na carga
      Vcabo = 30
      FP = 0.92 # FP da carga
      EFF = 1
      P = 4 # Potência ativa da carga
      aux2 = Fluxo (D, Vt,Vcabo,FP, P, EFF)
      b = b.append(aux2, ignore_index=True)
      c = c.append(aux2, ignore_index=True)
    P = (c['Preal'].sum())*3/1000000
    Vt = c['Vs'].mean()/1000
    FP = math.cos(math.atan(c['Pimag'].mean()/1000/c['Preal'].mean()/1000))
    D = dt.loc[34,'D'] # Distância
    aux3 =  Fluxo (D, Vt,Vcabo,FP, P, EFF)
    b = b.append(aux3, ignore_index=True)
    swg7x3 = b
    swg7x3
    P7x3x = swg7x3[swg7x3["Perd%"] < 3]['Perd%'].mean()
    P7x3y = swg7x3[swg7x3["Perd%"] > 3]['Perd%'].mean()
    P7x3 = (P7x3x + P7x3y)
    Ps7x3 = swg7x3[swg7x3['Ps'] > 8]['Ps'].sum()
    Pca7x3 = swg7x3['Custo_Aq'].sum()
    Pci7x3 = swg7x3['Custo_Ins'].sum()
    Pct7x3 = Pca7x3 + Pci7x3
    # Calculo da Confiabilidade
    P7x3RReq = dconf.loc[swg7x3[swg7x3["Perd%"] < 3]['Is%'].max(), 'R_Eq']**6
    P7x3RRum = dconf.loc[swg7x3[swg7x3["Perd%"] < 3]['Is%'].max(), 'R_Umb']
    P7x3RR = P7x3RReq*P7x3RRum
    P7x3RTeq = dconf.loc[swg7x3[swg7x3["Perd%"] > 3]['Is%'].max(), 'R_Eq']**3
    P7x3RTum = dconf.loc[swg7x3[swg7x3["Perd%"] > 3]['Is%'].max(), 'R_Umb']
    P7x3RT = P7x3RTeq*P7x3RTum
    P7x3R = P7x3RR*P7x3RT
    P7x3P = (1-P7x3RT)**2
    # Criação da Tabela
    dr7x3 = pd.DataFrame({'Tipo':['7x3'],
                          'Ps':[Ps7x3],
                          'Perd%':[P7x3],
                          'Cust_Aq':[Pca7x3],
                          'Custo_Ins':[Pci7x3],
                          'Custo_T':[Pct7x3],
                          'Conf_S%': [round(P7x3R,2)],
                          'Prob_10': [round(P7x3P,4)]})
    dr = dr.append(dr7x3, ignore_index=True)
    dr
  elif n == 4:
    for i in range(35,39):
      D = dt.loc[i,'D'] # Distância
      aux = Fluxo (D, Vt,Vcabo,FP, P, EFF)
      b = b.append(aux, ignore_index=True)
    P = (b['Preal'].sum())*3/1000000
    Vt = b['Vs'].mean()/1000
    FP = math.cos(math.atan(b['Pimag'].mean()/1000/b['Preal'].mean()/1000))
    D = dt.loc[39,'D'] # Distância
    aux1 =  Fluxo (D, Vt,Vcabo,FP, P, EFF)
    b = b.append(aux1, ignore_index=True)
    for i in range(40,46):
      D = dt.loc[i,'D'] # Distância
      Vt =  25 # Tensão na carga
      Vcabo = 30
      FP = 0.92 # FP da carga
      EFF = 1
      P = 4 # Potência ativa da carga
      aux2 = Fluxo (D, Vt,Vcabo,FP, P, EFF)
      b = b.append(aux2, ignore_index=True)
      c = c.append(aux2, ignore_index=True)
    P = (c['Preal'].sum())*3/1000000
    Vt = c['Vs'].mean()/1000
    FP = math.cos(math.atan(c['Pimag'].mean()/1000/c['Preal'].mean()/1000))
    D = dt.loc[46,'D'] # Distância
    aux3 =  Fluxo (D, Vt,Vcabo,FP, P, EFF)
    b = b.append(aux3, ignore_index=True)
    swg6x4 = b
    swg6x4
    P6x4x = swg6x4[swg6x4["Perd%"] < 3]['Perd%'].mean()
    P6x4y = swg6x4[swg6x4["Perd%"] > 3]['Perd%'].mean()
    P6x4 = (P6x4x + P6x4y)
    Ps6x4 = swg6x4[swg6x4['Ps'] > 8]['Ps'].sum()
    Pca6x4 = swg6x4['Custo_Aq'].sum()
    Pci6x4 = swg6x4['Custo_Ins'].sum()
    Pct6x4 = Pca6x4 + Pci6x4
    # Calculo da Confiabilidade
    P6x4RReq = dconf.loc[swg6x4[swg6x4["Perd%"] < 3]['Is%'].max(), 'R_Eq']**6
    P6x4RRum = dconf.loc[swg6x4[swg6x4["Perd%"] < 3]['Is%'].max(), 'R_Umb']
    P6x4RR = P6x4RReq*P6x4RRum
    P6x4RTeq = dconf.loc[swg6x4[swg6x4["Perd%"] > 3]['Is%'].max(), 'R_Eq']**3
    P6x4RTum = dconf.loc[swg6x4[swg6x4["Perd%"] > 3]['Is%'].max(), 'R_Umb']
    P6x4RT = P6x4RTeq*P6x4RTum
    P6x4R = P6x4RR*P6x4RT
    P6x4P = (1-P6x4RT)**2
    # Criação da Tabela
    dr6x4 = pd.DataFrame({'Tipo':['6x4'],
                          'Ps':[Ps6x4],
                          'Perd%':[P6x4],
                          'Cust_Aq':[Pca6x4],
                          'Custo_Ins':[Pci6x4],
                          'Custo_T':[Pct6x4],
                          'Conf_S%': [round(P6x4R,2)],
                          'Prob_10': [round(P6x4P,4)]})
    dr = dr.append(dr6x4, ignore_index=True)
    dr
  elif n == 5:
    for i in range(47,52):
      D = dt.loc[i,'D'] # Distância
      aux = Fluxo (D, Vt,Vcabo,FP, P, EFF)
      b = b.append(aux, ignore_index=True)
    P = (b['Preal'].sum())*3/1000000
    Vt = b['Vs'].mean()/1000
    FP = math.cos(math.atan(b['Pimag'].mean()/1000/b['Preal'].mean()/1000))
    D = dt.loc[52,'D'] # Distância
    aux1 =  Fluxo (D, Vt,Vcabo,FP, P, EFF)
    b = b.append(aux1, ignore_index=True)
    for i in range(53,58):
      D = dt.loc[i,'D'] # Distância
      Vt =  25 # Tensão na carga
      Vcabo = 30
      FP = 0.92 # FP da carga
      EFF = 1
      P = 4 # Potência ativa da carga
      aux2 = Fluxo (D, Vt,Vcabo,FP, P, EFF)
      b = b.append(aux2, ignore_index=True)
      c = c.append(aux2, ignore_index=True)
    P = (c['Preal'].sum())*3/1000000
    Vt = c['Vs'].mean()/1000
    FP = math.cos(math.atan(c['Pimag'].mean()/1000/c['Preal'].mean()/1000))
    D = dt.loc[58,'D'] # Distância
    print(D)
    aux3 =  Fluxo (D, Vt,Vcabo,FP, P, EFF)
    b = b.append(aux3, ignore_index=True)  
    swg5x5 = b
    swg5x5
    P5x5x = swg5x5[swg5x5["Perd%"] < 3]['Perd%'].mean()
    P5x5y = swg5x5[swg5x5["Perd%"] > 3]['Perd%'].mean()
    P5x5 = (P5x5x + P5x5y)
    Ps5x5 = swg5x5[swg5x5['Ps'] > 8]['Ps'].sum()
    Pca5x5 = swg5x5['Custo_Aq'].sum()
    Pci5x5 = swg5x5['Custo_Ins'].sum()
    Pct5x5 = Pca5x5 + Pci5x5
    # Calculo da Confiabilidade
    P5x5RReq = dconf.loc[swg5x5[swg5x5["Perd%"] < 3]['Is%'].max(), 'R_Eq']**6
    P5x5RRum = dconf.loc[swg5x5[swg5x5["Perd%"] < 3]['Is%'].max(), 'R_Umb']
    P5x5RR = P5x5RReq*P5x5RRum
    P5x5RTeq = dconf.loc[swg5x5[swg5x5["Perd%"] > 3]['Is%'].max(), 'R_Eq']**3
    P5x5RTum = dconf.loc[swg5x5[swg5x5["Perd%"] > 3]['Is%'].max(), 'R_Umb']
    P5x5RT = P5x5RTeq*P5x5RTum
    P5x5R = P5x5RR*P5x5RT
    P5x5P = (1-P5x5RT)**2
    # Criação da Tabela
    dr5x5 = pd.DataFrame({'Tipo':['5x5'],
                          'Ps':[Ps5x5],
                          'Perd%':[P5x5],
                          'Cust_Aq':[Pca5x5],
                          'Custo_Ins':[Pci5x5],
                          'Custo_T':[Pct5x5],
                          'Conf_S%': [round(P5x5R,2)],
                          'Prob_10': [round(P5x5P,4)]})
    dr = dr.append(dr5x5, ignore_index=True)
    dr
  elif n == 6:
    for i in range(59,65):
      D = dt.loc[i,'D'] # Distância
      aux = Fluxo (D, Vt,Vcabo,FP, P, EFF)
      b = b.append(aux, ignore_index=True)
    P = (b['Preal'].sum())*3/1000000
    Vt = b['Vs'].mean()/1000
    FP = math.cos(math.atan(b['Pimag'].mean()/1000/b['Preal'].mean()/1000))
    D = dt.loc[65,'D'] # Distância
    aux1 =  Fluxo (D, Vt,Vcabo,FP, P, EFF)
    b = b.append(aux1, ignore_index=True)
    for i in range(66,68):
      D = dt.loc[i,'D'] # Distância
      Vt =  25 # Tensão na carga
      Vcabo = 30
      FP = 0.92 # FP da carga
      EFF = 1
      P = 4 # Potência ativa da carga
      aux2 = Fluxo (D, Vt,Vcabo,FP, P, EFF)
      b = b.append(aux2, ignore_index=True)
      c = c.append(aux2, ignore_index=True)
    P = (c['Preal'].sum())*3/1000000
    Vt = c['Vs'].mean()/1000
    FP = math.cos(math.atan(c['Pimag'].mean()/1000/c['Preal'].mean()/1000))
    D = dt.loc[68,'D'] # Distância

    #print(D)
    aux3 =  Fluxo (D, Vt,Vcabo,FP, P, EFF)
    b = b.append(aux3, ignore_index=True)  
    for i in range(69,71):
      D = dt.loc[i,'D'] # Distância
      Vt =  25 # Tensão na carga
      Vcabo = 30
      FP = 0.92 # FP da carga
      EFF = 1
      P = 4 # Potência ativa da carga
      aux4 = Fluxo (D, Vt,Vcabo,FP, P, EFF)
      b = b.append(aux4, ignore_index=True)
      d = d.append(aux4, ignore_index=True)
    P = (d['Preal'].sum())*3/1000000
    Vt = d['Vs'].mean()/1000
    FP = math.cos(math.atan(d['Pimag'].mean()/1000/d['Preal'].mean()/1000))
    D = dt.loc[71,'D'] # Distância
    #print(D)
    aux5 =  Fluxo (D, Vt,Vcabo,FP, P, EFF)
    b = b.append(aux5, ignore_index=True) 
    swg2x2x6 = b
    swg2x2x6
    P2x2x6x = swg2x2x6[swg2x2x6["Perd%"] < 3]['Perd%'].mean()
    P2x2x6y = swg2x2x6[swg2x2x6["Perd%"] > 3]['Perd%'].mean()
    P2x2x6 = (P2x2x6x + P2x2x6y)
    Ps2x2x6 = swg2x2x6[swg2x2x6['Ps'] > 8]['Ps'].sum()
    Pca2x2x6 = swg2x2x6['Custo_Aq'].sum()
    Pci2x2x6 = swg2x2x6['Custo_Ins'].sum()
    Pct2x2x6 = Pca2x2x6 + Pci2x2x6
    # Calculo da Confiabilidade
    P2x2x6RReq = dconf.loc[swg2x2x6[swg2x2x6["Perd%"] < 3]['Is%'].max(), 'R_Eq']**6
    P2x2x6RRum = dconf.loc[swg2x2x6[swg2x2x6["Perd%"] < 3]['Is%'].max(), 'R_Umb']
    P2x2x6RR = P2x2x6RReq*P2x2x6RRum
    P2x2x6RTeq = dconf.loc[swg2x2x6[swg2x2x6["Perd%"] > 3]['Is%'].max(), 'R_Eq']**3
    P2x2x6RTum = dconf.loc[swg2x2x6[swg2x2x6["Perd%"] > 3]['Is%'].max(), 'R_Umb']
    P2x2x6RT = P2x2x6RTeq*P2x2x6RTum
    P2x2x6R = P2x2x6RR*P2x2x6RT
    P2x2x6P = (1-P2x2x6RT)**3
    # Criação da Tabela
    dr2x2x6 = pd.DataFrame({'Tipo':['2x2x6'],
                          'Ps':[Ps2x2x6],
                          'Perd%':[P2x2x6],
                          'Cust_Aq':[Pca2x2x6],
                          'Custo_Ins':[Pci2x2x6],
                          'Custo_T':[Pct2x2x6],
                          'Conf_S%': [round(P2x2x6R,2)],
                          'Prob_10': [round(P2x2x6P,4)]})
    dr = dr.append(dr2x2x6, ignore_index=True)
    dr
  elif n == 7:
    for i in range(72,77):
      D = dt.loc[i,'D'] # Distância
      aux = Fluxo (D, Vt,Vcabo,FP, P, EFF)
      b = b.append(aux, ignore_index=True)
    P = (b['Preal'].sum())*3/1000000
    Vt = b['Vs'].mean()/1000
    FP = math.cos(math.atan(b['Pimag'].mean()/1000/b['Preal'].mean()/1000))
    D = dt.loc[77,'D'] # Distância
    aux1 =  Fluxo (D, Vt,Vcabo,FP, P, EFF)
    b = b.append(aux1, ignore_index=True)
    for i in range(78,80):
      D = dt.loc[i,'D'] # Distância
      Vt =  25 # Tensão na carga
      Vcabo = 30
      FP = 0.92 # FP da carga
      EFF = 1
      P = 4 # Potência ativa da carga
      aux2 = Fluxo (D, Vt,Vcabo,FP, P, EFF)
      b = b.append(aux2, ignore_index=True)
      c = c.append(aux2, ignore_index=True)
    P = (c['Preal'].sum())*3/1000000
    Vt = c['Vs'].mean()/1000
    FP = math.cos(math.atan(c['Pimag'].mean()/1000/c['Preal'].mean()/1000))
    D = dt.loc[80,'D'] # Distância
    #print(D)
    aux3 =  Fluxo (D, Vt,Vcabo,FP, P, EFF)
    b = b.append(aux3, ignore_index=True)  
    for i in range(81,84):
      D = dt.loc[i,'D'] # Distância
      Vt =  25 # Tensão na carga
      Vcabo = 30
      FP = 0.92 # FP da carga
      EFF = 1
      P = 4 # Potência ativa da carga
      aux4 = Fluxo (D, Vt,Vcabo,FP, P, EFF)
      b = b.append(aux4, ignore_index=True)
      d = d.append(aux4, ignore_index=True)
    P = (d['Preal'].sum())*3/1000000
    Vt = d['Vs'].mean()/1000
    FP = math.cos(math.atan(d['Pimag'].mean()/1000/d['Preal'].mean()/1000))
    D = dt.loc[84,'D'] # Distância
    #print(D)
    aux5 =  Fluxo (D, Vt,Vcabo,FP, P, EFF)
    b = b.append(aux5, ignore_index=True) 
    swg2x3x5 = b
    swg2x3x5
    P2x3x5x = swg2x3x5[swg2x3x5["Perd%"] < 3]['Perd%'].mean()
    P2x3x5y = swg2x3x5[swg2x3x5["Perd%"] > 3]['Perd%'].mean()
    P2x3x5 = (P2x3x5x + P2x3x5y)
    Ps2x3x5 = swg2x3x5[swg2x3x5['Ps'] > 8]['Ps'].sum()
    Pca2x3x5 = swg2x3x5['Custo_Aq'].sum()
    Pci2x3x5 = swg2x3x5['Custo_Ins'].sum()
    Pct2x3x5 = Pca2x3x5 + Pci2x3x5
    # Calculo da Confiabilidade
    P2x3x5RReq = dconf.loc[swg2x3x5[swg2x3x5["Perd%"] < 3]['Is%'].max(), 'R_Eq']**6
    P2x3x5RRum = dconf.loc[swg2x3x5[swg2x3x5["Perd%"] < 3]['Is%'].max(), 'R_Umb']
    P2x3x5RR = P2x3x5RReq*P2x3x5RRum
    P2x3x5RTeq = dconf.loc[swg2x3x5[swg2x3x5["Perd%"] > 3]['Is%'].max(), 'R_Eq']**3
    P2x3x5RTum = dconf.loc[swg2x3x5[swg2x3x5["Perd%"] > 3]['Is%'].max(), 'R_Umb']
    P2x3x5RT = P2x3x5RTeq*P2x3x5RTum
    P2x3x5R = P2x3x5RR*P2x3x5RT
    P2x3x5P = (1-P2x3x5RT)**3
    # Criação da Tabela
    dr2x3x5 = pd.DataFrame({'Tipo':['2x3x5'],
                          'Ps':[Ps2x3x5],
                          'Perd%':[P2x3x5],
                          'Cust_Aq':[Pca2x3x5],
                          'Custo_Ins':[Pci2x3x5],
                          'Custo_T':[Pct2x3x5],
                          'Conf_S%': [round(P2x3x5R,2)],
                          'Prob_10': [round(P2x3x5P,4)]})
    dr = dr.append(dr2x3x5, ignore_index=True)
    dr
  elif n == 8:
    for i in range(85,89):
      D = dt.loc[i,'D'] # Distância
      aux = Fluxo (D, Vt,Vcabo,FP, P, EFF)
      b = b.append(aux, ignore_index=True)
    P = (b['Preal'].sum())*3/1000000
    Vt = b['Vs'].mean()/1000
    FP = math.cos(math.atan(b['Pimag'].mean()/1000/b['Preal'].mean()/1000))
    D = dt.loc[89,'D'] # Distância
    aux1 =  Fluxo (D, Vt,Vcabo,FP, P, EFF)
    b = b.append(aux1, ignore_index=True)
    for i in range(90,94):
      D = dt.loc[i,'D'] # Distância
      Vt =  25 # Tensão na carga
      Vcabo = 30
      FP = 0.92 # FP da carga
      EFF = 1
      P = 4 # Potência ativa da carga
      aux2 = Fluxo (D, Vt,Vcabo,FP, P, EFF)
      b = b.append(aux2, ignore_index=True)
      c = c.append(aux2, ignore_index=True)
    P = (c['Preal'].sum())*3/1000000
    Vt = c['Vs'].mean()/1000
    FP = math.cos(math.atan(c['Pimag'].mean()/1000/c['Preal'].mean()/1000))
    D = dt.loc[94,'D'] # Distância
    #print(D)
    aux3 =  Fluxo (D, Vt,Vcabo,FP, P, EFF)
    b = b.append(aux3, ignore_index=True)  
    for i in range(95,97):
      D = dt.loc[i,'D'] # Distância
      Vt =  25 # Tensão na carga
      Vcabo = 30
      FP = 0.92 # FP da carga
      EFF = 1
      P = 4 # Potência ativa da carga
      aux4 = Fluxo (D, Vt,Vcabo,FP, P, EFF)
      b = b.append(aux4, ignore_index=True)
      d = d.append(aux4, ignore_index=True)
    P = (d['Preal'].sum())*3/1000000
    Vt = d['Vs'].mean()/1000
    FP = math.cos(math.atan(d['Pimag'].mean()/1000/d['Preal'].mean()/1000))
    D = dt.loc[97,'D'] # Distância
    #print(D)
    aux5 =  Fluxo (D, Vt,Vcabo,FP, P, EFF)
    b = b.append(aux5, ignore_index=True) 
    swg2x4x4 = b
    swg2x4x4
    P2x4x4x = swg2x4x4[swg2x4x4["Perd%"] < 3]['Perd%'].mean()
    P2x4x4y = swg2x4x4[swg2x4x4["Perd%"] > 3]['Perd%'].mean()
    P2x4x4 = (P2x4x4x + P2x4x4y)
    Ps2x4x4 = swg2x4x4[swg2x4x4['Ps'] > 8]['Ps'].sum()
    Pca2x4x4 = swg2x4x4['Custo_Aq'].sum()
    Pci2x4x4 = swg2x4x4['Custo_Ins'].sum()
    Pct2x4x4 = Pca2x4x4 + Pci2x4x4
    # Calculo da Confiabilidade
    P2x4x4RReq = dconf.loc[swg2x4x4[swg2x4x4["Perd%"] < 3]['Is%'].max(), 'R_Eq']**6
    P2x4x4RRum = dconf.loc[swg2x4x4[swg2x4x4["Perd%"] < 3]['Is%'].max(), 'R_Umb']
    P2x4x4RR = P2x4x4RReq*P2x4x4RRum
    P2x4x4RTeq = dconf.loc[swg2x4x4[swg2x4x4["Perd%"] > 3]['Is%'].max(), 'R_Eq']**3
    P2x4x4RTum = dconf.loc[swg2x4x4[swg2x4x4["Perd%"] > 3]['Is%'].max(), 'R_Umb']
    P2x4x4RT = P2x4x4RTeq*P2x4x4RTum
    P2x4x4R = P2x4x4RR*P2x4x4RT
    P2x4x4P = (1-P2x4x4RT)**3
    # Criação da Tabela
    dr2x4x4 = pd.DataFrame({'Tipo':['2x4x4'],
                          'Ps':[Ps2x4x4],
                          'Perd%':[P2x4x4],
                          'Cust_Aq':[Pca2x4x4],
                          'Custo_Ins':[Pci2x4x4],
                          'Custo_T':[Pct2x4x4],
                          'Conf_S%': [round(P2x4x4R,2)],
                          'Prob_10': [round(P2x4x4P,4)]})
    dr = dr.append(dr2x4x4, ignore_index=True)
    dr
  elif n == 9:
    for i in range(98,102):
      D = dt.loc[i,'D'] # Distância
      aux = Fluxo (D, Vt,Vcabo,FP, P, EFF)
      b = b.append(aux, ignore_index=True)
    P = (b['Preal'].sum())*3/1000000
    Vt = b['Vs'].mean()/1000
    FP = math.cos(math.atan(b['Pimag'].mean()/1000/b['Preal'].mean()/1000))
    D = dt.loc[102,'D'] # Distância
    aux1 =  Fluxo (D, Vt,Vcabo,FP, P, EFF)
    b = b.append(aux1, ignore_index=True)
    for i in range(103,106):
      D = dt.loc[i,'D'] # Distância
      Vt =  25 # Tensão na carga
      Vcabo = 30
      FP = 0.92 # FP da carga
      EFF = 1
      P = 4 # Potência ativa da carga
      aux2 = Fluxo (D, Vt,Vcabo,FP, P, EFF)
      b = b.append(aux2, ignore_index=True)
      c = c.append(aux2, ignore_index=True)
    P = (c['Preal'].sum())*3/1000000
    Vt = c['Vs'].mean()/1000
    FP = math.cos(math.atan(c['Pimag'].mean()/1000/c['Preal'].mean()/1000))
    D = dt.loc[106,'D'] # Distância
    #print(D)
    aux3 =  Fluxo (D, Vt,Vcabo,FP, P, EFF)
    b = b.append(aux3, ignore_index=True)  
    for i in range(107,110):
      D = dt.loc[i,'D'] # Distância
      Vt =  25 # Tensão na carga
      Vcabo = 30
      FP = 0.92 # FP da carga
      EFF = 1
      P = 4 # Potência ativa da carga
      aux4 = Fluxo (D, Vt,Vcabo,FP, P, EFF)
      b = b.append(aux4, ignore_index=True)
      d = d.append(aux4, ignore_index=True)
    P = (d['Preal'].sum())*3/1000000
    Vt = d['Vs'].mean()/1000
    FP = math.cos(math.atan(d['Pimag'].mean()/1000/d['Preal'].mean()/1000))
    D = dt.loc[110,'D'] # Distância
    #print(D)
    aux5 =  Fluxo (D, Vt,Vcabo,FP, P, EFF)
    b = b.append(aux5, ignore_index=True) 
    swg3x3x4 = b
    swg3x3x4
    P3x3x4x = swg3x3x4[swg3x3x4["Perd%"] < 3]['Perd%'].mean()
    P3x3x4y = swg3x3x4[swg3x3x4["Perd%"] > 3]['Perd%'].mean()
    P3x3x4 = (P3x3x4x + P3x3x4y)
    Ps3x3x4 = swg3x3x4[swg3x3x4['Ps'] > 8]['Ps'].sum()
    Pca3x3x4 = swg3x3x4['Custo_Aq'].sum()
    Pci3x3x4 = swg3x3x4['Custo_Ins'].sum()
    Pct3x3x4 = Pca3x3x4 + Pci3x3x4
    # Calculo da Confiabilidade
    P3x3x4RReq = dconf.loc[swg3x3x4[swg3x3x4["Perd%"] < 3]['Is%'].max(), 'R_Eq']**6
    P3x3x4RRum = dconf.loc[swg3x3x4[swg3x3x4["Perd%"] < 3]['Is%'].max(), 'R_Umb']
    P3x3x4RR = P3x3x4RReq*P3x3x4RRum
    P3x3x4RTeq = dconf.loc[swg3x3x4[swg3x3x4["Perd%"] > 3]['Is%'].max(), 'R_Eq']**3
    P3x3x4RTum = dconf.loc[swg3x3x4[swg3x3x4["Perd%"] > 3]['Is%'].max(), 'R_Umb']
    P3x3x4RT = P3x3x4RTeq*P3x3x4RTum
    P3x3x4R = P3x3x4RR*P3x3x4RT
    P3x3x4P = (1-P3x3x4RT)**3
    # Criação da Tabela
    dr3x3x4 = pd.DataFrame({'Tipo':['3x3x4'],
                          'Ps':[Ps3x3x4],
                          'Perd%':[P3x3x4],
                          'Cust_Aq':[Pca3x3x4],
                          'Custo_Ins':[Pci3x3x4],
                          'Custo_T':[Pct3x3x4],
                          'Conf_S%': [round(P3x3x4R,2)],
                          'Prob_10': [round(P3x3x4P,4)]})
    dr = dr.append(dr3x3x4, ignore_index=True)
    dr
  elif n == 10:
    for i in range(111,115):
      D = dt.loc[i,'D'] # Distância
      aux = Fluxo (D, Vt,Vcabo,FP, P, EFF)
      b = b.append(aux, ignore_index=True)
    P = (b['Preal'].sum())*3/1000000
    Vt = b['Vs'].mean()/1000
    FP = math.cos(math.atan(b['Pimag'].mean()/1000/b['Preal'].mean()/1000))
    D = dt.loc[115,'D'] # Distância
    aux1 =  Fluxo (D, Vt,Vcabo,FP, P, EFF)
    b = b.append(aux1, ignore_index=True)
    for i in range(116,118):
      D = dt.loc[i,'D'] # Distância
      Vt =  25 # Tensão na carga
      Vcabo = 30
      FP = 0.92 # FP da carga
      EFF = 1
      P = 4 # Potência ativa da carga
      aux2 = Fluxo (D, Vt,Vcabo,FP, P, EFF)
      b = b.append(aux2, ignore_index=True)
      c = c.append(aux2, ignore_index=True)
    P = (c['Preal'].sum())*3/1000000
    Vt = c['Vs'].mean()/1000
    FP = math.cos(math.atan(c['Pimag'].mean()/1000/c['Preal'].mean()/1000))
    D = dt.loc[118,'D'] # Distância
    #print(D)
    aux3 =  Fluxo (D, Vt,Vcabo,FP, P, EFF)
    b = b.append(aux3, ignore_index=True)  
    for i in range(119,121):
      D = dt.loc[i,'D'] # Distância
      Vt =  25 # Tensão na carga
      Vcabo = 30
      FP = 0.92 # FP da carga
      EFF = 1
      P = 4 # Potência ativa da carga
      aux4 = Fluxo (D, Vt,Vcabo,FP, P, EFF)
      b = b.append(aux4, ignore_index=True)
      d = d.append(aux4, ignore_index=True)
    P = (d['Preal'].sum())*3/1000000
    Vt = d['Vs'].mean()/1000
    FP = math.cos(math.atan(d['Pimag'].mean()/1000/d['Preal'].mean()/1000))
    D = dt.loc[121,'D'] # Distância
    #print(D)
    aux5 =  Fluxo (D, Vt,Vcabo,FP, P, EFF)
    b = b.append(aux5, ignore_index=True)
    for i in range(122,124):
      D = dt.loc[i,'D'] # Distância
      Vt =  25 # Tensão na carga
      Vcabo = 30
      FP = 0.92 # FP da carga
      EFF = 1
      P = 4 # Potência ativa da carga
      aux6 = Fluxo (D, Vt,Vcabo,FP, P, EFF)
      b = b.append(aux6, ignore_index=True)
      e = e.append(aux6, ignore_index=True)
    P = (e['Preal'].sum())*3/1000000
    Vt = e['Vs'].mean()/1000
    FP = math.cos(math.atan(e['Pimag'].mean()/1000/e['Preal'].mean()/1000))
    D = dt.loc[124,'D'] # Distância
    #print(D)
    aux7 =  Fluxo (D, Vt,Vcabo,FP, P, EFF)
    b = b.append(aux7, ignore_index=True) 
    swg2x2x2x4 = b
    swg2x2x2x4
    P2x2x2x4x = swg2x2x2x4[swg2x2x2x4["Perd%"] < 3]['Perd%'].mean()
    P2x2x2x4y = swg2x2x2x4[swg2x2x2x4["Perd%"] > 3]['Perd%'].mean()
    P2x2x2x4 = (P2x2x2x4x + P2x2x2x4y)
    Ps2x2x2x4 = swg2x2x2x4[swg2x2x2x4['Ps'] > 8]['Ps'].sum()
    Pca2x2x2x4 = swg2x2x2x4['Custo_Aq'].sum()
    Pci2x2x2x4 = swg2x2x2x4['Custo_Ins'].sum()
    Pct2x2x2x4 = Pca2x2x2x4 + Pci2x2x2x4
    # Calculo da Confiabilidade
    P2x2x2x4RReq = dconf.loc[swg2x2x2x4[swg2x2x2x4["Perd%"] < 3]['Is%'].max(), 'R_Eq']**6
    P2x2x2x4RRum = dconf.loc[swg2x2x2x4[swg2x2x2x4["Perd%"] < 3]['Is%'].max(), 'R_Umb']
    P2x2x2x4RR = P2x2x2x4RReq*P2x2x2x4RRum
    P2x2x2x4RTeq = dconf.loc[swg2x2x2x4[swg2x2x2x4["Perd%"] > 3]['Is%'].max(), 'R_Eq']**3
    P2x2x2x4RTum = dconf.loc[swg2x2x2x4[swg2x2x2x4["Perd%"] > 3]['Is%'].max(), 'R_Umb']
    P2x2x2x4RT = P2x2x2x4RTeq*P2x2x2x4RTum
    P2x2x2x4R = P2x2x2x4RR*P2x2x2x4RT
    P2x2x2x4P = (1-P2x2x2x4RT)**4
    # Criação da Tabela
    dr2x2x2x4 = pd.DataFrame({'Tipo':['2x2x2x4'],
                          'Ps':[Ps2x2x2x4],
                          'Perd%':[P2x2x2x4],
                          'Cust_Aq':[Pca2x2x2x4],
                          'Custo_Ins':[Pci2x2x2x4],
                          'Custo_T':[Pct2x2x2x4],
                          'Conf_S%': [round(P2x2x2x4R,2)],
                          'Prob_10': [round(P2x2x2x4P,4)]})
    dr = dr.append(dr2x2x2x4, ignore_index=True)
    dr
  elif n == 11:
    for i in range(125,128):
      D = dt.loc[i,'D'] # Distância
      aux = Fluxo (D, Vt,Vcabo,FP, P, EFF)
      b = b.append(aux, ignore_index=True)
    P = (b['Preal'].sum())*3/1000000
    Vt = b['Vs'].mean()/1000
    FP = math.cos(math.atan(b['Pimag'].mean()/1000/b['Preal'].mean()/1000))
    D = dt.loc[128,'D'] # Distância
    aux1 =  Fluxo (D, Vt,Vcabo,FP, P, EFF)
    b = b.append(aux1, ignore_index=True)
    for i in range(129,132):
      D = dt.loc[i,'D'] # Distância
      Vt =  25 # Tensão na carga
      Vcabo = 30
      FP = 0.92 # FP da carga
      EFF = 1
      P = 4 # Potência ativa da carga
      aux2 = Fluxo (D, Vt,Vcabo,FP, P, EFF)
      b = b.append(aux2, ignore_index=True)
      c = c.append(aux2, ignore_index=True)
    P = (c['Preal'].sum())*3/1000000
    Vt = c['Vs'].mean()/1000
    FP = math.cos(math.atan(c['Pimag'].mean()/1000/c['Preal'].mean()/1000))
    D = dt.loc[132,'D'] # Distância
    #print(D)
    aux3 =  Fluxo (D, Vt,Vcabo,FP, P, EFF)
    b = b.append(aux3, ignore_index=True)  
    for i in range(133,135):
      D = dt.loc[i,'D'] # Distância
      Vt =  25 # Tensão na carga
      Vcabo = 30
      FP = 0.92 # FP da carga
      EFF = 1
      P = 4 # Potência ativa da carga
      aux4 = Fluxo (D, Vt,Vcabo,FP, P, EFF)
      b = b.append(aux4, ignore_index=True)
      d = d.append(aux4, ignore_index=True)
    P = (d['Preal'].sum())*3/1000000
    Vt = d['Vs'].mean()/1000
    FP = math.cos(math.atan(d['Pimag'].mean()/1000/d['Preal'].mean()/1000))
    D = dt.loc[135,'D'] # Distância
    #print(D)
    aux5 =  Fluxo (D, Vt,Vcabo,FP, P, EFF)
    b = b.append(aux5, ignore_index=True)
    for i in range(136,138):
      D = dt.loc[i,'D'] # Distância
      Vt =  25 # Tensão na carga
      Vcabo = 30
      FP = 0.92 # FP da carga
      EFF = 1
      P = 4 # Potência ativa da carga
      aux6 = Fluxo (D, Vt,Vcabo,FP, P, EFF)
      b = b.append(aux6, ignore_index=True)
      e = e.append(aux6, ignore_index=True)
    P = (e['Preal'].sum())*3/1000000
    Vt = e['Vs'].mean()/1000
    FP = math.cos(math.atan(e['Pimag'].mean()/1000/e['Preal'].mean()/1000))
    D = dt.loc[138,'D'] # Distância
    #print(D)
    aux7 =  Fluxo (D, Vt,Vcabo,FP, P, EFF)
    b = b.append(aux7, ignore_index=True) 
    swg2x2x3x3 = b
    swg2x2x3x3
    P2x2x3x3x = swg2x2x3x3[swg2x2x3x3["Perd%"] < 3]['Perd%'].mean()
    P2x2x3x3y = swg2x2x3x3[swg2x2x3x3["Perd%"] > 3]['Perd%'].mean()
    P2x2x3x3 = (P2x2x3x3x + P2x2x3x3y)
    Ps2x2x3x3 = swg2x2x3x3[swg2x2x3x3['Ps'] > 8]['Ps'].sum()
    Pca2x2x3x3 = swg2x2x3x3['Custo_Aq'].sum()
    Pci2x2x3x3 = swg2x2x3x3['Custo_Ins'].sum()
    Pct2x2x3x3 = Pca2x2x3x3 + Pci2x2x3x3
    # Calculo da Confiabilidade
    P2x2x3x3RReq = dconf.loc[swg2x2x3x3[swg2x2x3x3["Perd%"] < 3]['Is%'].max(), 'R_Eq']**6
    P2x2x3x3RRum = dconf.loc[swg2x2x3x3[swg2x2x3x3["Perd%"] < 3]['Is%'].max(), 'R_Umb']
    P2x2x3x3RR = P2x2x3x3RReq*P2x2x3x3RRum
    P2x2x3x3RTeq = dconf.loc[swg2x2x3x3[swg2x2x3x3["Perd%"] > 3]['Is%'].max(), 'R_Eq']**3
    P2x2x3x3RTum = dconf.loc[swg2x2x3x3[swg2x2x3x3["Perd%"] > 3]['Is%'].max(), 'R_Umb']
    P2x2x3x3RT = P2x2x3x3RTeq*P2x2x3x3RTum
    P2x2x3x3R = P2x2x3x3RR*P2x2x3x3RT
    P2x2x3x3P = (1-P2x2x3x3RT)**4
    # Criação da Tabela
    dr2x2x3x3 = pd.DataFrame({'Tipo':['2x2x3x3'],
                          'Ps':[Ps2x2x3x3],
                          'Perd%':[P2x2x3x3],
                          'Cust_Aq':[Pca2x2x3x3],
                          'Custo_Ins':[Pci2x2x3x3],
                          'Custo_T':[Pct2x2x3x3],
                          'Conf_S%': [round(P2x2x3x3R,2)],
                          'Prob_10': [round(P2x2x3x3P,4)]})
    dr = dr.append(dr2x2x3x3, ignore_index=True)
    dr
  elif n == 12:
    for i in range(139,141):
      D = dt.loc[i,'D'] # Distância
      aux = Fluxo (D, Vt,Vcabo,FP, P, EFF)
      b = b.append(aux, ignore_index=True)
    P = (b['Preal'].sum())*3/1000000
    Vt = b['Vs'].mean()/1000
    FP = math.cos(math.atan(b['Pimag'].mean()/1000/b['Preal'].mean()/1000))
    D = dt.loc[141,'D'] # Distância
    aux1 =  Fluxo (D, Vt,Vcabo,FP, P, EFF)
    b = b.append(aux1, ignore_index=True)
    for i in range(142,144):
      D = dt.loc[i,'D'] # Distância
      Vt =  25 # Tensão na carga
      Vcabo = 30
      FP = 0.92 # FP da carga
      EFF = 1
      P = 4 # Potência ativa da carga
      aux2 = Fluxo (D, Vt,Vcabo,FP, P, EFF)
      b = b.append(aux2, ignore_index=True)
      c = c.append(aux2, ignore_index=True)
    P = (c['Preal'].sum())*3/1000000
    Vt = c['Vs'].mean()/1000
    FP = math.cos(math.atan(c['Pimag'].mean()/1000/c['Preal'].mean()/1000))
    D = dt.loc[144,'D'] # Distância
    #print(D)
    aux3 =  Fluxo (D, Vt,Vcabo,FP, P, EFF)
    b = b.append(aux3, ignore_index=True)  
    for i in range(145,147):
      D = dt.loc[i,'D'] # Distância
      Vt =  25 # Tensão na carga
      Vcabo = 30
      FP = 0.92 # FP da carga
      EFF = 1
      P = 4 # Potência ativa da carga
      aux4 = Fluxo (D, Vt,Vcabo,FP, P, EFF)
      b = b.append(aux4, ignore_index=True)
      d = d.append(aux4, ignore_index=True)
    P = (d['Preal'].sum())*3/1000000
    Vt = d['Vs'].mean()/1000
    FP = math.cos(math.atan(d['Pimag'].mean()/1000/d['Preal'].mean()/1000))
    D = dt.loc[147,'D'] # Distância
    #print(D)
    aux5 =  Fluxo (D, Vt,Vcabo,FP, P, EFF)
    b = b.append(aux5, ignore_index=True)
    for i in range(148,150):
      D = dt.loc[i,'D'] # Distância
      Vt =  25 # Tensão na carga
      Vcabo = 30
      FP = 0.92 # FP da carga
      EFF = 1
      P = 4 # Potência ativa da carga
      aux6 = Fluxo (D, Vt,Vcabo,FP, P, EFF)
      b = b.append(aux6, ignore_index=True)
      e = e.append(aux6, ignore_index=True)
    P = (e['Preal'].sum())*3/1000000
    Vt = e['Vs'].mean()/1000
    FP = math.cos(math.atan(e['Pimag'].mean()/1000/e['Preal'].mean()/1000))
    D = dt.loc[150,'D'] # Distância
    #print(D)
    aux7 =  Fluxo (D, Vt,Vcabo,FP, P, EFF)
    b = b.append(aux7, ignore_index=True) 
    for i in range(151,153):
      D = dt.loc[i,'D'] # Distância
      Vt =  25 # Tensão na carga
      Vcabo = 30
      FP = 0.92 # FP da carga
      EFF = 1
      P = 4 # Potência ativa da carga
      aux8 = Fluxo (D, Vt,Vcabo,FP, P, EFF)
      b = b.append(aux8, ignore_index=True)
      f = f.append(aux8, ignore_index=True)
    P = (f['Preal'].sum())*3/1000000
    Vt = f['Vs'].mean()/1000
    FP = math.cos(math.atan(f['Pimag'].mean()/1000/f['Preal'].mean()/1000))
    D = dt.loc[153,'D'] # Distância
    #print(D)
    aux9 =  Fluxo (D, Vt,Vcabo,FP, P, EFF)
    b = b.append(aux9, ignore_index=True) 
    swg2x2x2x2x2 = b
    swg2x2x2x2x2
    # Calculo das Perdas
    P2x2x2x2x2x = swg2x2x2x2x2[swg2x2x2x2x2["Perd%"] < 3]['Perd%'].mean()
    P2x2x2x2x2y = swg2x2x2x2x2[swg2x2x2x2x2["Perd%"] > 3]['Perd%'].mean()
    P2x2x2x2x2 = (P2x2x2x2x2x + P2x2x2x2x2y)
    Ps2x2x2x2x2 = swg2x2x2x2x2[swg2x2x2x2x2['Ps'] > 8]['Ps'].sum()
    # Calculo do Custo
    Pca2x2x2x2x2 = swg2x2x2x2x2['Custo_Aq'].sum()
    Pci2x2x2x2x2 = swg2x2x2x2x2['Custo_Ins'].sum()
    Pct2x2x2x2x2 = Pca2x2x2x2x2 + Pci2x2x2x2x2    
    # Calculo da Confiabilidade
    P2x2x2x2x2RReq = dconf.loc[swg2x2x2x2x2[swg2x2x2x2x2["Perd%"] < 3]['Is%'].max(), 'R_Eq']**6
    P2x2x2x2x2RRum = dconf.loc[swg2x2x2x2x2[swg2x2x2x2x2["Perd%"] < 3]['Is%'].max(), 'R_Umb']
    P2x2x2x2x2RR = P2x2x2x2x2RReq*P2x2x2x2x2RRum
    P2x2x2x2x2RTeq = dconf.loc[swg2x2x2x2x2[swg2x2x2x2x2["Perd%"] > 3]['Is%'].max(), 'R_Eq']**3
    P2x2x2x2x2RTum = dconf.loc[swg2x2x2x2x2[swg2x2x2x2x2["Perd%"] > 3]['Is%'].max(), 'R_Umb']
    P2x2x2x2x2RT = P2x2x2x2x2RTeq*P2x2x2x2x2RTum
    P2x2x2x2x2R = P2x2x2x2x2RR*P2x2x2x2x2RT
    P2x2x2x2x2P = (1-P2x2x2x2x2RT)**5   
    # Criação da Tabela
    dr2x2x2x2x2 = pd.DataFrame({'Tipo':['2x2x2x2x2'],
                          'Ps':[Ps2x2x2x2x2],
                          'Perd%':[P2x2x2x2x2],
                          'Cust_Aq':[Pca2x2x2x2x2],
                          'Custo_Ins':[Pci2x2x2x2x2],
                          'Custo_T':[Pct2x2x2x2x2],
                          'Conf_S%': [round(P2x2x2x2x2R,2)],
                          'Prob_10': [round(P2x2x2x2x2P,4)]})
    dr = dr.append(dr2x2x2x2x2, ignore_index=True)
    dr
  b

29.89600926766594


In [123]:
swg7x3['Is%'].max()

95.0

In [122]:
swg7x3

,D,Ps,Bitola,num,Vs,Vs%,Is,Is%,Ir,Ir%,Vr,Vr%,Perdas,Perd%,Preal,Pimag,Custo_Aq,Custo_Ins
0,1.189826,4.35,35,1,25117.35,100.47,100.08,88.0,100.41,88.0,25000.00,100.0,0.02,0.48,1.339770e+06,557952.310290,0.34,1.100
1,0.486863,4.35,35,1,25048.06,100.19,100.27,88.0,100.41,88.0,25000.00,100.0,0.01,0.20,1.335972e+06,563902.831141,0.14,1.100
2,1.028778,4.35,35,1,25101.48,100.41,100.12,88.0,100.41,88.0,25000.00,100.0,0.02,0.42,1.338901e+06,559319.553937,0.29,1.100
3,29.717770,12.62,240,1,26178.54,104.34,278.28,88.0,277.16,88.0,25088.96,100.0,0.55,4.58,4.198554e+06,-249272.728657,14.86,3.575
4,3.151890,4.36,35,1,25310.15,101.24,99.55,87.0,100.41,88.0,25000.00,100.0,0.05,1.27,1.350293e+06,541104.792225,0.89,1.100
5,2.221354,4.36,35,1,25218.82,100.88,99.80,88.0,100.41,88.0,25000.00,100.0,0.04,0.90,1.345316e+06,549138.799264,0.63,1.100
6,1.390831,4.35,35,1,25137.14,100.55,100.02,88.0,100.41,88.0,25000.00,100.0,0.02,0.56,1.340853e+06,556242.520751,0.39,1.100
7,0.966650,4.35,35,1,25095.36,100.38,100.14,88.0,100.41,88.0,25000.00,100.0,0.02,0.39,1.338566e+06,559846.371214,0.27,1.100
8,1.390831,4.35,35,1,25137.14,100.55,100.02,88.0,100.41,88.0,25000.00,100.0,0.02,0.56,1.340853e+06,556242.519225,0.39,1.100
9,2.221354,4.36,35,1,25218.82,100.88,99.80,88.0,100.41,88.0,25000.00,100.0,0.04,0.90,1.345316e+06,549138.797337,0.63,1.100


In [133]:
dr

,Tipo,Ps,Perd%,Cust_Aq,Custo_Ins,Custo_T,Conf_S%,Prob_10
0,10,42.12,5.118000,56.51,20.075,76.585,0.12,0.7616
1,8x2,42.00,5.528000,56.35,20.900,77.250,0.12,0.5800
2,7x3,41.86,4.700000,55.30,21.450,76.750,0.22,0.3177
3,6x4,42.03,4.863000,51.95,20.900,72.850,0.15,0.5066
4,5x5,42.34,5.456000,57.64,23.100,80.740,0.22,0.3177
5,2x2x6,42.40,6.030000,55.36,23.100,78.460,0.15,0.3605
6,2x3x5,42.34,5.595667,55.87,23.650,79.520,0.22,0.1790
7,2x4x4,41.99,5.100667,50.33,21.725,72.055,0.17,0.2893
8,3x3x4,41.95,4.777000,50.63,22.000,72.630,0.17,0.2893
9,2x2x2x4,42.37,5.866000,53.86,23.925,77.785,0.17,0.1914


In [67]:
dconf = pd.read_excel("Conf.xlsx")
dconf

,I%,R_Eq,R_Umb
0,0.0,0.999969,0.999988
1,1.0,0.999939,0.999939
2,2.0,0.999939,0.999939
3,3.0,0.999939,0.999939
4,4.0,0.999939,0.999939
...,...,...,...
96,96.0,0.749300,0.922918
97,97.0,0.718498,0.912848
98,98.0,0.684218,0.899950
99,99.0,0.646194,0.883648


In [70]:
teste = dconf.loc[96, 'R_Eq']
teste

0.7493000000000001

In [44]:
dd = pd.read_excel("dados.xlsx")
dd

,ITEM,EQUIP,BLU,SPEC45,SWG,RISER,TR_TOP,TOTAL EQUIP
0,1,10,21,13,1,1,2,491.108333
1,2,8X2,22,13,2,2,3,501.748333
2,3,7X3,22,13,2,2,3,509.390833
3,4,6X4,22,13,2,2,3,509.208333
4,5,5X5,22,14,2,2,2,526.383333
5,6,2X2X6,23,14,3,3,4,536.983333
6,7,2X3X5,23,14,3,3,3,536.820833
7,8,2X4X4,23,13,3,3,3,519.483333
8,9,3X3X4,23,13,3,3,3,519.848333
9,10,2X2X2X4,24,14,4,4,4,547.258333


In [50]:
dd = pd.read_excel("dados.xlsx")
dd['TOTAL UMB'] = dr["Custo_T"]
dd['TOTAL/10'] = (dr["Custo_T"]+dd['TOTAL EQUIP'])/10
dd['Ps'] = dr['Ps']
dd['Perd%'] = dr["Perd%"]
dd

,ITEM,EQUIP,BLU,SPEC45,SWG,RISER,TR_TOP,TOTAL EQUIP,TOTAL UMB,TOTAL/10,Ps,Perd%
0,1,10,21,13,1,1,2,491.108333,75.075,56.618333,42.12,5.118000
1,2,8X2,22,13,2,2,3,501.748333,75.900,57.764833,42.00,5.528000
2,3,7X3,22,13,2,2,3,509.390833,76.450,58.584083,41.86,4.700000
3,4,6X4,22,13,2,2,3,509.208333,71.900,58.110833,42.03,4.863000
4,5,5X5,22,14,2,2,2,526.383333,81.100,60.748333,42.34,5.456000
5,6,2X2X6,23,14,3,3,4,536.983333,77.100,61.408333,42.40,6.030000
6,7,2X3X5,23,14,3,3,3,536.820833,78.650,61.547083,42.34,5.595667
7,8,2X4X4,23,13,3,3,3,519.483333,68.725,58.820833,41.99,5.100667
8,9,3X3X4,23,13,3,3,3,519.848333,70.000,58.984833,41.95,4.777000
9,10,2X2X2X4,24,14,4,4,4,547.258333,74.925,62.218333,42.37,5.866000


In [12]:
rq = pd.DataFrame()
for i in range(0,12):
  aux11 = pd.DataFrame({'BLU':[dd.loc[i,'BLU']**2],
                        'SPEC45':[dd.loc[i,'SPEC45']**2],
                        'SWG':[dd.loc[i,'SWG']**2],
                        'RISER':[dd.loc[i,'RISER']**2],
                        'TR_TOP':[dd.loc[i,'TR_TOP']**2],
                        'TOTAL EQUIP':[dd.loc[i,'TOTAL EQUIP']**2],
                        'TOTAL_UMB_TRANS':[dd.loc[i,'TOTAL_UMB_TRANS']**2],
                        'TOTAL_UMB_RAM':[dd.loc[i,'TOTAL_UMB_RAM']**2],
                        'TOTAL/10':[dd.loc[i,'TOTAL/10']**2],
                        'Ps':[dd.loc[i,'Ps']**2],
                        'Perd%':[dd.loc[i,'Perd%']**2]})
  rq = rq.append(aux11)
res = pd.DataFrame({'BLU':[(rq['BLU'].sum(0))**0.5],
                    'SPEC45':[(rq['SPEC45'].sum(0))**0.5],
                    'SWG':[(rq['SWG'].sum(0))**0.5],
                    'RISER':[(rq['RISER'].sum(0))**0.5],
                    'TR_TOP':[(rq['TR_TOP'].sum(0))**0.5],
                    'TOTAL EQUIP':[(rq['TOTAL EQUIP'].sum(0))**0.5],
                    'TOTAL_UMB_TRANS':[(rq['TOTAL_UMB_TRANS'].sum(0))**0.5],
                    'TOTAL_UMB_RAM':[(rq['TOTAL_UMB_RAM'].sum(0))**0.5],
                    'TOTAL/10':[(rq['TOTAL/10'].sum(0))**0.5],
                    'Ps':[(rq['Ps'].sum(0))**0.5],
                    'Perd%':[(rq['Perd%'].sum(0))**0.5],})
res


,BLU,SPEC45,SWG,RISER,TR_TOP,TOTAL EQUIP,TOTAL_UMB_TRANS,TOTAL_UMB_RAM,TOTAL/10,Ps,Perd%
0,79.183332,47.106263,10.488088,10.488088,11.61895,1826.217396,231.19223,51.27027,210.642104,142.091817,18.468274


In [13]:
n = 3

A = np.ones([n,n])
for i in range(0,n):
    for j in range(0,n):
        if i<j:
            aij = input('Quão importante é a opção{} da opção{} ?: '.format(i,j))
            A[i,j] = float(aij)
            A[j,i] = round(1/float(aij),1)

Quão importante é a opção0 da opção1 ?: 0.33
Quão importante é a opção0 da opção2 ?: 1
Quão importante é a opção1 da opção2 ?: 3


In [43]:
FAHP_CRIT = pd.DataFrame({'Equip':[fuzzy_AHP(A)[0]],
                          'Custo':[fuzzy_AHP(A)[1]],
                          'Elec':[fuzzy_AHP(A)[2]]})
FAHP_CRIT

1.0452458129355322
1.0452458129355322
1.0452458129355322


,Equip,Custo,Elec
0,0.201589,0.596821,0.201589


In [15]:
n = 5

B = np.ones([n,n])
for i in range(0,n):
    for j in range(0,n):
        if i<j:
            aij = input('Quão importante é a opção{} da opção{} ?: '.format(i,j))
            B[i,j] = float(aij)
            B[j,i] = round(1/float(aij),1)

Quão importante é a opção0 da opção1 ?: 0.5
Quão importante é a opção0 da opção2 ?: 0.5
Quão importante é a opção0 da opção3 ?: 0.5
Quão importante é a opção0 da opção4 ?: 0.33
Quão importante é a opção1 da opção2 ?: 2
Quão importante é a opção1 da opção3 ?: 0.2
Quão importante é a opção1 da opção4 ?: 0.33
Quão importante é a opção2 da opção3 ?: 1
Quão importante é a opção2 da opção4 ?: 0.2
Quão importante é a opção3 da opção4 ?: 0.2


In [16]:
FAHP_EQUIP = pd.DataFrame({'BLU':[fuzzy_AHP(B)[0]],
                           'SPEC45':[fuzzy_AHP(B)[1]],
                           'SWG':[fuzzy_AHP(B)[2]],
                           'RISER':[fuzzy_AHP(B)[3]],
                           'TR_TOP':[fuzzy_AHP(B)[4]]})
FAHP_EQUIP

[[0.0505537816079895, 0.08646766486899136, 0.18513999620444552], [0.06829613874849216, 0.12534001012548648, 0.21773926576634609], [0.07234089841398049, 0.11833193822504837, 0.20077909960337023], [0.11891048411690022, 0.18754348334360818, 0.2873086836645753], [0.29626093821816435, 0.48231690343686573, 0.7582121618583083]]
[[0.0505537816079895, 0.08646766486899136, 0.18513999620444552], [0.06829613874849216, 0.12534001012548648, 0.21773926576634609], [0.07234089841398049, 0.11833193822504837, 0.20077909960337023], [0.11891048411690022, 0.18754348334360818, 0.2873086836645753], [0.29626093821816435, 0.48231690343686573, 0.7582121618583083]]
[[0.0505537816079895, 0.08646766486899136, 0.18513999620444552], [0.06829613874849216, 0.12534001012548648, 0.21773926576634609], [0.07234089841398049, 0.11833193822504837, 0.20077909960337023], [0.11891048411690022, 0.18754348334360818, 0.2873086836645753], [0.29626093821816435, 0.48231690343686573, 0.7582121618583083]]
[[0.0505537816079895, 0.0864676

,BLU,SPEC45,SWG,RISER,TR_TOP
0,0.098958,0.126362,0.120242,0.182385,0.472054


In [17]:
n = 4

C = np.ones([n,n])
for i in range(0,n):
    for j in range(0,n):
        if i<j:
            aij = input('Quão importante é a opção{} da opção{} ?: '.format(i,j))
            C[i,j] = float(aij)
            C[j,i] = round(1/float(aij),1)

Quão importante é a opção0 da opção1 ?: 0.5
Quão importante é a opção0 da opção2 ?: 2
Quão importante é a opção0 da opção3 ?: 3
Quão importante é a opção1 da opção2 ?: 1
Quão importante é a opção1 da opção3 ?: 0.5
Quão importante é a opção2 da opção3 ?: 2


In [33]:
FAHP_CUSTO = pd.DataFrame({'TOTAL EQUIP':[fuzzy_AHP(C)[0]],
                           'TOTAL_UMB_TRANS':[fuzzy_AHP(C)[1]],
                           'TOTAL_UMB_RAM':[fuzzy_AHP(C)[2]],
                           'TOTAL/10':[fuzzy_AHP(C)[3]]})
FAHP_CUSTO

[[0.16135635778607268, 0.32289086615516055, 0.6286684820693899], [0.13568398284070432, 0.24534400267564302, 0.44453574678951907], [0.13568398284070432, 0.24534400267564302, 0.44453574678951907], [0.09594306436506118, 0.18642112849355336, 0.3738085159274395]]
[[0.16135635778607268, 0.32289086615516055, 0.6286684820693899], [0.13568398284070432, 0.24534400267564302, 0.44453574678951907], [0.13568398284070432, 0.24534400267564302, 0.44453574678951907], [0.09594306436506118, 0.18642112849355336, 0.3738085159274395]]
[[0.16135635778607268, 0.32289086615516055, 0.6286684820693899], [0.13568398284070432, 0.24534400267564302, 0.44453574678951907], [0.13568398284070432, 0.24534400267564302, 0.44453574678951907], [0.09594306436506118, 0.18642112849355336, 0.3738085159274395]]
[[0.16135635778607268, 0.32289086615516055, 0.6286684820693899], [0.13568398284070432, 0.24534400267564302, 0.44453574678951907], [0.13568398284070432, 0.24534400267564302, 0.44453574678951907], [0.09594306436506118, 0.1864

,TOTAL EQUIP,TOTAL_UMB_TRANS,TOTAL_UMB_RAM,TOTAL/10
0,0.325393,0.241378,0.241378,0.191851


In [19]:
n = 2

D = np.ones([n,n])
for i in range(0,n):
    for j in range(0,n):
        if i<j:
            aij = input('Quão importante é a opção{} da opção{} ?: '.format(i,j))
            D[i,j] = float(aij)
            D[j,i] = round(1/float(aij),1)

Quão importante é a opção0 da opção1 ?: 2


In [41]:
FAHP_ELET = pd.DataFrame({'Ps':[fuzzy_AHP(D)[0]],
                          'Perd%':[fuzzy_AHP(D)[1]]})
FAHP_ELET

,Ps,Perd%
0,0.643853,0.356147


In [21]:
FAHP_PESO = pd.DataFrame({'BLU':[FAHP_CRIT.loc[0, 'Equip']*FAHP_EQUIP.loc[0, 'BLU']],
                         'SPEC45': [FAHP_CRIT.loc[0, 'Equip']*FAHP_EQUIP.loc[0, 'SPEC45']],
                         'SWG': [FAHP_CRIT.loc[0, 'Equip']*FAHP_EQUIP.loc[0, 'SWG']],
                         'RISER':[FAHP_CRIT.loc[0, 'Equip']*FAHP_EQUIP.loc[0, 'RISER']],
                         'TR_TOP':[FAHP_CRIT.loc[0, 'Equip']*FAHP_EQUIP.loc[0, 'TR_TOP']],
                         'TOTAL EQUIP':[FAHP_CRIT.loc[0, 'Custo']*FAHP_CUSTO.loc[0, 'TOTAL EQUIP']],
                         'TOTAL_UMB_TRANS':[FAHP_CRIT.loc[0, 'Custo']*FAHP_CUSTO.loc[0, 'TOTAL_UMB_TRANS']],
                         'TOTAL_UMB_RAM':[FAHP_CRIT.loc[0, 'Custo']*FAHP_CUSTO.loc[0,  'TOTAL_UMB_RAM']],
                         'TOTAL/10':[FAHP_CRIT.loc[0, 'Custo']*FAHP_CUSTO.loc[0,  'TOTAL/10']],
                         'Ps':[FAHP_CRIT.loc[0, 'Elec']*FAHP_ELET.loc[0,  'Ps']],
                         'Perd%':[FAHP_CRIT.loc[0, 'Elec']*FAHP_ELET.loc[0,  'Perd%']]})
FAHP_PESO

,BLU,SPEC45,SWG,RISER,TR_TOP,TOTAL EQUIP,TOTAL_UMB_TRANS,TOTAL_UMB_RAM,TOTAL/10,Ps,Perd%
0,0.019949,0.025473,0.024239,0.036767,0.095161,0.194202,0.144059,0.144059,0.114501,0.129794,0.071795


In [22]:
nor = pd.DataFrame()
for i in range(0,12):
  aux12 = dd.iloc[i,2:]/res
  nor = nor.append(aux12, ignore_index = True)
nor

,BLU,SPEC45,SWG,RISER,TR_TOP,TOTAL EQUIP,TOTAL_UMB_TRANS,TOTAL_UMB_RAM,TOTAL/10,Ps,Perd%
0,0.265207,0.275972,0.0953463,0.0953463,0.172133,0.268921,0.305215,0.360499,0.275422,0.296428,0.277124
1,0.277836,0.275972,0.190693,0.190693,0.258199,0.274747,0.293384,0.318104,0.278143,0.295584,0.299324
2,0.277836,0.275972,0.190693,0.190693,0.258199,0.278932,0.252707,0.310292,0.277116,0.294598,0.25449
3,0.277836,0.275972,0.190693,0.190693,0.258199,0.278832,0.284869,0.291483,0.280102,0.295795,0.263316
4,0.277836,0.2972,0.190693,0.190693,0.172133,0.288237,0.181631,0.287204,0.27682,0.297976,0.295426
5,0.290465,0.2972,0.286039,0.286039,0.344265,0.294041,0.304587,0.287349,0.295351,0.178054,0.277909
6,0.290465,0.2972,0.286039,0.286039,0.258199,0.293952,0.246189,0.272305,0.288498,0.298399,0.326506
7,0.290465,0.275972,0.286039,0.286039,0.258199,0.284459,0.300359,0.271033,0.286182,0.297976,0.302988
8,0.290465,0.275972,0.286039,0.286039,0.258199,0.284659,0.291029,0.26154,0.2851,0.295513,0.276185
9,0.303094,0.2972,0.381385,0.381385,0.344265,0.299668,0.320077,0.266899,0.301431,0.295232,0.25866


In [23]:
norpe = pd.DataFrame()
for i in range(0,12):
  aux13 = nor.iloc[i,]*FAHP_PESO
  norpe = norpe.append(aux13, ignore_index = True)
norpe

,BLU,SPEC45,SWG,RISER,TR_TOP,TOTAL EQUIP,TOTAL_UMB_TRANS,TOTAL_UMB_RAM,TOTAL/10,Ps,Perd%
0,0.00529058,0.00702987,0.00231114,0.00350559,0.0163803,0.0522249,0.0439691,0.0519332,0.0315361,0.0384746,0.0198962
1,0.00554251,0.00702987,0.00462228,0.00701118,0.0245705,0.0533564,0.0422647,0.0458258,0.0318476,0.0383649,0.0214901
2,0.00554251,0.00702987,0.00462228,0.00701118,0.0245705,0.0541691,0.0364049,0.0447005,0.0317301,0.0382371,0.0182712
3,0.00554251,0.00702987,0.00462228,0.00701118,0.0245705,0.0541497,0.041038,0.0419908,0.0320719,0.0383924,0.0189049
4,0.00554251,0.00757063,0.00462228,0.00701118,0.0163803,0.0559761,0.0261656,0.0413744,0.0316962,0.0386755,0.0212102
5,0.00579445,0.00757063,0.00693342,0.0105168,0.0327606,0.0571033,0.0438786,0.0413954,0.033818,0.0231103,0.0199526
6,0.00579445,0.00757063,0.00693342,0.0105168,0.0245705,0.057086,0.0354658,0.0392281,0.0330333,0.0387303,0.0234416
7,0.00579445,0.00702987,0.00693342,0.0105168,0.0245705,0.0552424,0.0432695,0.0390448,0.0327681,0.0386755,0.0217532
8,0.00579445,0.00702987,0.00693342,0.0105168,0.0245705,0.0552812,0.0419254,0.0376773,0.0326442,0.0383558,0.0198288
9,0.00604638,0.00757063,0.00924456,0.0140224,0.0327606,0.058196,0.0461101,0.0384493,0.0345142,0.0383193,0.0185706


In [24]:
norpe['BLU'].min()

0.0052905819653780144

In [25]:
best = pd.DataFrame({'BLU':[norpe['BLU'].min()],
                        'SPEC45':[norpe['SPEC45'].min()],
                        'SWG':[norpe['SWG'].min()],
                        'RISER':[norpe['RISER'].min()],
                        'TR_TOP':[norpe['TR_TOP'].min()],
                        'TOTAL EQUIP':[norpe['TOTAL EQUIP'].min()],
                        'TOTAL_UMB_TRANS':[norpe['TOTAL_UMB_TRANS'].min()],
                        'TOTAL_UMB_RAM':[norpe['TOTAL_UMB_RAM'].min()],
                        'TOTAL/10':[norpe['TOTAL/10'].min()],
                        'Ps':[norpe['Ps'].min()],
                        'Perd%':[norpe['Perd%'].min()]})
best

,BLU,SPEC45,SWG,RISER,TR_TOP,TOTAL EQUIP,TOTAL_UMB_TRANS,TOTAL_UMB_RAM,TOTAL/10,Ps,Perd%
0,0.005291,0.00703,0.002311,0.003506,0.01638,0.052225,0.026166,0.037082,0.031536,0.02311,0.018271


In [26]:
worst = pd.DataFrame({'BLU':[norpe['BLU'].max()],
                        'SPEC45':[norpe['SPEC45'].max()],
                        'SWG':[norpe['SWG'].max()],
                        'RISER':[norpe['RISER'].max()],
                        'TR_TOP':[norpe['TR_TOP'].max()],
                        'TOTAL EQUIP':[norpe['TOTAL EQUIP'].max()],
                        'TOTAL_UMB_TRANS':[norpe['TOTAL_UMB_TRANS'].max()],
                        'TOTAL_UMB_RAM':[norpe['TOTAL_UMB_RAM'].max()],
                        'TOTAL/10':[norpe['TOTAL/10'].max()],
                        'Ps':[norpe['Ps'].max()],
                        'Perd%':[norpe['Perd%'].max()]})
worst

,BLU,SPEC45,SWG,RISER,TR_TOP,TOTAL EQUIP,TOTAL_UMB_TRANS,TOTAL_UMB_RAM,TOTAL/10,Ps,Perd%
0,0.006298,0.008111,0.011556,0.017528,0.040951,0.06115,0.048951,0.051933,0.03626,0.03873,0.023442


In [27]:
dbest = pd.DataFrame()
for i in range(0,12):
  aux14 = (norpe.iloc[i,]-best)**2
  dbest = dbest.append(aux14, ignore_index = True)
dbest

,BLU,SPEC45,SWG,RISER,TR_TOP,TOTAL EQUIP,TOTAL_UMB_TRANS,TOTAL_UMB_RAM,TOTAL/10,Ps,Perd%
0,0,0,0,0,0,0,0.000316966,0.000220565,0,0.00023606,2.64054e-06
1,6.347e-08,0,5.34137e-06,1.22891e-05,6.70786e-05,1.28022e-06,0.000259183,7.64586e-05,9.70527e-08,0.000232704,1.0361e-05
2,6.347e-08,0,5.34137e-06,1.22891e-05,6.70786e-05,3.77983e-06,0.000104843,5.80454e-05,3.7637e-08,0.000228819,0
3,6.347e-08,0,5.34137e-06,1.22891e-05,6.70786e-05,3.70475e-06,0.00022119,2.40989e-05,2.87124e-07,0.000233541,4.01528e-07
4,6.347e-08,2.92421e-07,5.34137e-06,1.22891e-05,0,1.40713e-05,0,1.84267e-05,2.56322e-08,0.000242276,8.63743e-06
5,2.5388e-07,2.92421e-07,2.13655e-05,4.91566e-05,0.000268314,2.37987e-05,0.00031375,1.8607e-05,5.20712e-06,0,2.82692e-06
6,2.5388e-07,2.92421e-07,2.13655e-05,4.91566e-05,6.70786e-05,2.36304e-05,8.64937e-05,4.60667e-06,2.24176e-06,0.000243985,2.67328e-05
7,2.5388e-07,0,2.13655e-05,4.91566e-05,6.70786e-05,9.10485e-06,0.000292545,3.85361e-06,1.51793e-06,0.000242276,1.21237e-05
8,2.5388e-07,0,2.13655e-05,4.91566e-05,6.70786e-05,9.3406e-06,0.000248373,3.54601e-07,1.22805e-06,0.000232425,2.42609e-06
9,5.7123e-07,2.92421e-07,4.80724e-05,0.000110602,0.000268314,3.56534e-05,0.000397783,1.87027e-06,8.86916e-06,0.000231313,8.96029e-08


In [28]:
dworst = pd.DataFrame()
for i in range(0,12):
  aux15 = (norpe.iloc[i,]-worst)**2
  dworst = dworst.append(aux15, ignore_index = True)
dworst

,BLU,SPEC45,SWG,RISER,TR_TOP,TOTAL EQUIP,TOTAL_UMB_TRANS,TOTAL_UMB_RAM,TOTAL/10,Ps,Perd%
0,1.01552e-06,1.16968e-06,8.5462e-05,0.000196626,0.000603707,7.96497e-05,2.4815e-05,0,2.23179e-05,6.54164e-08,1.25699e-05
1,5.7123e-07,1.16968e-06,4.80724e-05,0.000110602,0.000268314,6.07339e-05,4.47007e-05,3.73e-05,1.94715e-05,1.33503e-07,3.80845e-06
2,5.7123e-07,1.16968e-06,4.80724e-05,0.000110602,0.000268314,4.87272e-05,0.000157395,5.23116e-05,2.05225e-05,2.43309e-07,2.67328e-05
3,5.7123e-07,1.16968e-06,4.80724e-05,0.000110602,0.000268314,4.89985e-05,6.26087e-05,9.88507e-05,1.75422e-05,1.14228e-07,2.05818e-05
4,5.7123e-07,2.92421e-07,4.80724e-05,0.000110602,0.000603707,2.6765e-05,0.000519157,0.000111488,2.08309e-05,3.00382e-09,4.97926e-06
5,2.5388e-07,2.92421e-07,2.13655e-05,4.91566e-05,6.70786e-05,1.63723e-05,2.57255e-05,0.000111046,5.9647e-06,0.000243985,1.21733e-05
6,2.5388e-07,2.92421e-07,2.13655e-05,4.91566e-05,0.000268314,1.65125e-05,0.00018184,0.00016142,1.04131e-05,0,0
7,2.5388e-07,1.16968e-06,2.13655e-05,4.91566e-05,0.000268314,3.48955e-05,3.22745e-05,0.00016611,1.21951e-05,3.00382e-09,2.85093e-06
8,2.5388e-07,1.16968e-06,2.13655e-05,4.91566e-05,0.000268314,3.44384e-05,4.93528e-05,0.000203232,1.30755e-05,1.40261e-07,1.30522e-05
9,6.347e-08,2.92421e-07,5.34137e-06,1.22891e-05,6.70786e-05,8.72387e-06,8.06862e-06,0.000181814,3.04876e-06,1.68965e-07,2.3727e-05


In [29]:
(dworst.iloc[0,].sum())**0.5/((dbest.iloc[0,].sum())**0.5+(dworst.iloc[0,].sum())**0.5)

0.5349844645321952

In [30]:
drank = pd.DataFrame()
for i in range(0,12):
  aux16 = pd.DataFrame({'EQUIP':[dd.loc[i,'EQUIP']],'Rank':[(dworst.iloc[i,].sum())**0.5/((dbest.iloc[i,].sum())**0.5+(dworst.iloc[i,].sum())**0.5)]})
  drank = drank.append(aux16, ignore_index = True)
drank

,EQUIP,Rank
0,10,0.534984
1,8X2,0.486102
2,7X3,0.552927
3,6X4,0.522009
4,5X5,0.686581
5,2X2X6,0.470028
6,2X3X5,0.537388
7,2X4X4,0.478474
8,3X3X4,0.504191
9,2X2X2X4,0.346647


In [31]:
drank.sort_values(by = 'Rank', ascending = False, ignore_index = True)

,EQUIP,Rank
0,5X5,0.686581
1,7X3,0.552927
2,2X3X5,0.537388
3,10,0.534984
4,6X4,0.522009
5,3X3X4,0.504191
6,8X2,0.486102
7,2X4X4,0.478474
8,2X2X6,0.470028
9,2X2X3X3,0.357878
